In [1]:
import turtle
import random
import time

In [2]:
class PolygonGA:
    def __init__(self, population_size, target_sides):
        self.population_size = population_size
        self.target = target_sides
        self.target_bin = f'{target_sides:08b}'
        self.population = []
        self.generation = 0
        self.history = []

    def generate_initial_population(self):
        self.population.clear()
        while len(self.population) < self.population_size:
            rand = random.randint(3, 15)
            binary = f'{rand:08b}'
            if binary not in self.population:
                self.population.append(binary)

    def fitness(self, chromosome):
        return sum(1 for a, b in zip(chromosome, self.target_bin) if a == b)

    def crossover(self, parent1, parent2):
        pivot = random.randint(1, len(parent1) - 2)
        child = parent1[:pivot] + parent2[pivot:]
        return child

    def mutate(self, chromosome, rate=0.1):
        if random.random() < rate:
            index = random.randint(0, len(chromosome) - 1)
            new_bit = '1' if chromosome[index] == '0' else '0'
            chromosome = chromosome[:index] + new_bit + chromosome[index+1:]
        return chromosome

    def evolve(self):
        self.generation += 1
        print(f"\nGeneration {self.generation}")

        fitness_scores = [(ch, self.fitness(ch)) for ch in self.population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)

        # Draw current generation
        self.draw_generation([int(ch, 2) for ch, _ in fitness_scores])

        if fitness_scores[0][0] == self.target_bin:
            print("🎯 Perfect polygon evolved!")
            return True

        new_population = [ch for ch, _ in fitness_scores[:2]]  # Elites

        while len(new_population) < self.population_size:
            parents = random.sample(fitness_scores[:6], 2)
            child = self.crossover(parents[0][0], parents[1][0])
            child = self.mutate(child)
            new_population.append(child)

        self.population = new_population
        return False

    def draw_generation(self, sides_list):
        screen = turtle.Screen()
        screen.clear()
        screen.title(f"Polygon Evolution - Generation {self.generation}")
        screen.bgcolor("black")
        screen.colormode(255)

        t = turtle.Turtle()
        t.hideturtle()
        t.speed(0)

        positions = [(-250, 150), (0, 150), (250, 150),
                     (-250, -100), (0, -100), (250, -100)]

        for i, sides in enumerate(sides_list[:min(len(positions), len(sides_list))]):
            x, y = positions[i]
            t.penup()
            t.goto(x, y)
            t.pendown()
            color = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
            t.pencolor(color)
            self.draw_polygon(t, sides)

        screen.update()
        time.sleep(1)

    def draw_polygon(self, turtle_obj, sides):
        if sides < 3:
            return
        angle = 360 / sides
        size = 20 if sides <= 6 else 15
        turtle_obj.begin_fill()
        for _ in range(sides):
            turtle_obj.forward(size)
            turtle_obj.left(angle)
        turtle_obj.end_fill()

In [3]:
population_size = int(input("Enter population size (>2): "))
while population_size < 3:
    population_size = int(input("Enter a valid population size (>2): "))

target_sides = int(input("Enter target polygon sides (3-15): "))
while target_sides not in range(3, 16):
    target_sides = int(input("Enter a value between 3 and 15: "))

ga = PolygonGA(population_size, target_sides)
ga.generate_initial_population()

while not ga.evolve():
    pass

turtle.done()



Generation 1

Generation 2

Generation 3
🎯 Perfect polygon evolved!
